In [15]:
import pandas as pd

# Ruta a tu archivo Parquet
hydro = 'core_ferc1__yearly_hydroelectric_plants_sched406.parquet'

try:
    # Leer el archivo Parquet en un DataFrame de Pandas
    df = pd.read_parquet(hydro)

    
    print("Archivo Parquet cargado exitosamente.")
    print("\nInformación del DataFrame:")
    df.info()

except FileNotFoundError:
    print(f"Error: El archivo '{hydro}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo Parquet: {e}")

Archivo Parquet cargado exitosamente.

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7421 entries, 0 to 7420
Data columns (total 37 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   record_id                               7421 non-null   object  
 1   utility_id_ferc1                        7421 non-null   int32   
 2   report_year                             7421 non-null   int32   
 3   plant_name_ferc1                        7421 non-null   object  
 4   project_num                             7389 non-null   float64 
 5   plant_type                              7004 non-null   category
 6   construction_type                       7098 non-null   category
 7   construction_year                       7143 non-null   float64 
 8   installation_year                       7143 non-null   float64 
 9   capacity_mw                             7354 non-nul

In [16]:
import pandas as pd

# Ruta a tu archivo Parquet
ruta_archivo_parquet = 'core_ferc1__yearly_steam_plants_sched402 (1).parquet'

try:
    # Leer el archivo Parquet en un DataFrame de Pandas
    df2 = pd.read_parquet(ruta_archivo_parquet)

    # Ahora puedes trabajar con el DataFrame
    print("Archivo Parquet cargado exitosamente.")
    print("\nInformación del DataFrame:")
    df2.info()

except FileNotFoundError:
    print(f"Error: El archivo '{ruta_archivo_parquet}' no se encontró.")
except Exception as e:
    print(f"Ocurrió un error al intentar leer el archivo Parquet: {e}")

Archivo Parquet cargado exitosamente.

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34111 entries, 0 to 34110
Data columns (total 39 columns):
 #   Column                                  Non-Null Count  Dtype   
---  ------                                  --------------  -----   
 0   record_id                               34111 non-null  object  
 1   utility_id_ferc1                        34111 non-null  int32   
 2   report_year                             34111 non-null  int32   
 3   plant_name_ferc1                        34111 non-null  object  
 4   plant_type                              32695 non-null  category
 5   construction_type                       28836 non-null  category
 6   construction_year                       32473 non-null  float64 
 7   installation_year                       32227 non-null  float64 
 8   capacity_mw                             33489 non-null  float32 
 9   peak_demand_mw                          28056 non-

In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML
from datetime import datetime
import re 
from ipywidgets import embed

# Configuración inicial
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Paleta de colores
COLOR_PALETTE = px.colors.qualitative.Plotly

## 1. Función para limpiar y preparar datos (MEJORADA)
def prepare_data(df):
    """
    Prepara los datos asegurando formatos correctos y valores consistentes
    """
    df = df.copy()
    
    # Convertir año a datetime (manejo de diferentes formatos)
    if not pd.api.types.is_datetime64_any_dtype(df['report_year']):
        try:
            # Primero intentar convertir a entero y luego a datetime
            df['report_year'] = pd.to_datetime(df['report_year'].astype(int), format='%Y')
        except:
            # Si falla, intentar convertir directamente
            df['report_year'] = pd.to_datetime(df['report_year'])
    
    # Asegurar costos positivos
    cost_cols = [col for col in df.columns if 'capex_' in col or 'opex_' in col]
    for col in cost_cols:
        if col in df.columns: # Añadir esta comprobación para evitar KeyError si la columna no existe
            df[col] = df[col].abs()
    
    # Calcular totales si no existen
    if 'capex_total' not in df.columns:
        capex_cols = [col for col in df.columns if col.startswith('capex_') and col != 'capex_total' and col in df.columns]
        df['capex_total'] = df[cape_cols].sum(axis=1)
    
    if 'opex_total' not in df.columns:
        opex_cols = [col for col in df.columns if col.startswith('opex_') and col != 'opex_total' and col in df.columns]
        df['opex_total'] = df[opex_cols].sum(axis=1)
    
    # Calcular ratios con protección contra división por cero
    df['capex_per_mw'] = df.apply(
        lambda x: x['capex_total'] / x['capacity_mw'] if x['capacity_mw'] > 0 else 0,
        axis=1
    )
    
    df['opex_per_mwh'] = df.apply(
        lambda x: x['opex_total'] / x['net_generation_mwh'] if x['net_generation_mwh'] > 0 else 0,
        axis=1
    )
    
    # Asegurar que plant_type sea string y manejar valores nulos
    df['plant_type'] = df['plant_type'].astype(str)
    df['plant_type'] = df['plant_type'].replace('nan', 'No especificado')
    
    # Asegurar que plant_name_ferc1 sea string y manejar valores nulos
    if 'plant_name_ferc1' in df.columns:
        df['plant_name_ferc1'] = df['plant_name_ferc1'].astype(str)
        df['plant_name_ferc1'] = df['plant_name_ferc1'].replace('nan', 'No especificado')
        
        # Nuevo: Limpiar plant_name_ferc1 de posibles números entre paréntesis como "(1)"
        # Esto es crucial para que el nombre base coincida con el filtro de planta
        # Usamos una función para manejar NaN y aplicar la expresión regular
        def clean_plant_name(name):
            if pd.isna(name) or name == 'No especificado':
                return 'No especificado'
            return re.sub(r'\s*\(\d+\)\s*', '', str(name)).strip()

        df['plant_name_ferc1_clean'] = df['plant_name_ferc1'].apply(clean_plant_name)
        
        # Crear columna combinada para mostrar en el selector, usando el nombre limpio
        df['plant_name_with_type'] = df['plant_name_ferc1_clean'] + ' (' + df['plant_type'] + ')'
    else:
        # Si no hay 'plant_name_ferc1', crear una columna dummy para evitar errores
        df['plant_name_ferc1'] = 'No disponible'
        df['plant_name_ferc1_clean'] = 'No disponible'
        df['plant_name_with_type'] = 'No disponible'
        
    # NUEVA FUNCIONALIDAD: Crear columna de rangos de capacidad
    # Definir los bins de forma que siempre sean crecientes y el último abarque el valor máximo
    min_capacity = df['capacity_mw'].min()
    max_capacity = df['capacity_mw'].max()
    
    # Bins predefinidos
    predefined_bins = [0, 10, 50, 100, 250, 500, 1000, 2000]
    
    # Asegurarse de que el primer bin sea 0 o el mínimo valor si el mínimo es negativo (aunque capacity_mw es positivo)
    # y que el último bin abarque el máximo valor.
    bins = sorted(list(set(predefined_bins + [0, max_capacity + 1]))) # Usar set para eliminar duplicados y sorted para ordenar
    
    # Asegurarse de que el último bin es mayor que el penúltimo
    if len(bins) >= 2 and bins[-1] <= bins[-2]:
        bins[-1] = bins[-2] + 1 # Asegurar monotonicidad si el último bin se solapa

    # Generar las etiquetas dinámicamente basándose en los bins finales
    labels = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i+1] - 1 if i < len(bins) - 2 else bins[i+1] # Para el último bin, no restamos 1
        
        if i == len(bins) - 2 and upper == bins[-1]: # Si es el último rango y cubre hasta el final
             labels.append(f'>{lower} MW')
        elif i == 0 and lower == 0 and upper == 10: # Caso específico para el primer rango
            labels.append(f'{lower}-{upper} MW')
        elif i < len(bins) - 2 : # Rangos intermedios
             labels.append(f'{lower+1}-{upper} MW')
        else: # Si el último bin es un valor intermedio pero abarca hasta el final del rango
            labels.append(f'{lower+1}-{upper} MW')


    # Corrección para el caso donde el último bin cubre el máximo
    # Reconstrucción más sencilla y robusta de labels
    labels = []
    for i in range(len(bins) - 1):
        start = bins[i]
        end = bins[i+1]
        if i == len(bins) - 2: # Último intervalo
            labels.append(f'>{start} MW')
        elif start == 0: # Primer intervalo
            labels.append(f'{start}-{end} MW')
        else: # Intervalos intermedios
            labels.append(f'{start+1}-{end} MW')
            
    # Última revisión para asegurar que labels tenga el tamaño correcto
    if len(labels) != len(bins) - 1:
        # Esto no debería pasar con la lógica anterior, pero es una salvaguarda
        print("Advertencia: El número de etiquetas no coincide con el número de bins. Ajustando etiquetas.")
        labels = [f'Rango {i}' for i in range(len(bins)-1)] # Fallback genérico

    # Crear la nueva columna 'capacity_mw_range'
    try:
        df['capacity_mw_range'] = pd.cut(df['capacity_mw'], bins=bins, labels=labels, right=True, include_lowest=True)
    except ValueError as e:
        print(f"Error al crear rangos de capacidad: {e}")
        print(f"Valores de bins: {bins}")
        print(f"Valores de labels: {labels}")
        print(f"Máxima capacidad: {max_capacity}")
        # En caso de error, se puede asignar un valor por defecto.
        df['capacity_mw_range'] = 'Error de cálculo de rango' 
    
    df['capacity_mw_range'] = df['capacity_mw_range'].astype(str).replace('nan', 'No especificado')
    
    return df

## 2. Evolución de CAPEX, OPEX y Capacidad (MEJORADA)
def plot_cost_evolution(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) # Asegurar que el dataframe que llega aquí ya esté limpio

    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]

    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Evolución de Costos.</p>
        </div>
        """))
        return # Salir de la función si no hay datos
        
    # Agrupar por año
    yearly_data = filtered_df.groupby('report_year').agg({
        'capex_total': 'sum',
        'opex_total': 'sum',
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }).reset_index()
    
    # Calcular ratios
    yearly_data['capex_per_mw'] = yearly_data['capex_total'] / yearly_data['capacity_mw'].replace(0, 1)
    yearly_data['opex_per_mwh'] = yearly_data['opex_total'] / yearly_data['net_generation_mwh'].replace(0, 1)
    
    # Crear figura
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # CAPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['capex_total'],
            name='CAPEX Total (USD)',
            marker_color=COLOR_PALETTE[0],
            opacity=0.7,
            hovertemplate='<b>CAPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['opex_total'],
            name='OPEX Total (USD)',
            marker_color=COLOR_PALETTE[1],
            opacity=0.7,
            hovertemplate='<b>OPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # Capacidad (Línea)
    fig.add_trace(
        go.Scatter(
            x=yearly_data['report_year'],
            y=yearly_data['capacity_mw'],
            name='Capacidad (MW)',
            line=dict(color=COLOR_PALETTE[2], width=3),
            mode='lines+markers',
            hovertemplate='<b>Capacidad</b><br>Año: %{x|%Y}<br>Total: %{y:,.0f} MW<extra></extra>'
        ),
        secondary_y=True
    )
    
    # Actualizar diseño
    fig.update_layout(
        title='Evolución de Costos y Capacidad',
        barmode='group',
        plot_bgcolor='white',
        hovermode='x unified',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.1, x=0.5, xanchor="center"),  # Leyenda centrada
        margin=dict(t=120, b=80),  # Margen superior aumentado para la leyenda
        xaxis=dict(
            tickformat='%Y',  # Mostrar solo el año
            type='date'  # Asegurar que se trate como fecha
        )
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="Costos (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Capacidad (MW)", secondary_y=True)
    fig.update_xaxes(title_text="Año")
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Evolución de Costos</h4>
        <p>Este gráfico muestra la evolución temporal de los costos de capital (CAPEX) y operación (OPEX), 
        junto con la capacidad instalada total. Los valores de costos se muestran en USD (eje izquierdo) 
        mientras que la capacidad se muestra en MW (eje derecho).</p>
        <p><b>Nota:</b> Todos los valores de costos han sido convertidos a valores absolutos para evitar 
        interpretaciones erróneas con números negativos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 3. Composición de Costos Detallada (MEJORADA)
def plot_cost_composition(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) # Asegurar que el dataframe que llega aquí ya esté limpio
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Composición de Costos.</p>
        </div>
        """))
        return # Salir de la función si no hay datos

    # Componentes CAPEX (excluyendo totales)
    capex_cols = [col for col in df_clean.columns 
                  if col.startswith('capex_') 
                  and col not in ['capex_total', 'capex_per_mw']
                  and df_clean[col].sum() > 0 and col in filtered_df.columns] # Asegurar que la columna existe en el filtered_df
    
    # Componentes OPEX (excluyendo totales)
    opex_cols = [col for col in df_clean.columns 
                 if col.startswith('opex_') 
                 and col not in ['opex_total', 'opex_per_mwh']
                 and df_clean[col].sum() > 0 and col in filtered_df.columns] # Asegurar que la columna existe en el filtered_df
    
    # Crear figura con subplots
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=("Composición CAPEX (USD)", "Composición OPEX (USD)"),
                        vertical_spacing=0.15)
    
    # ---- Gráfico CAPEX ----
    if capex_cols and not filtered_df.empty:
        capex_data = filtered_df.groupby('plant_type')[capex_cols].mean().reset_index()
        # Evitar división por cero en capex_pct
        sum_capex_by_plant_type = capex_data.set_index('plant_type').sum(axis=1).replace(0,1)
        capex_pct = capex_data.set_index('plant_type').div(sum_capex_by_plant_type, axis=0) * 100
        
        for i, col in enumerate(capex_cols):
            if not capex_data[col].empty:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                 zip(capex_data[col], capex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=capex_data['plant_type'],
                        y=capex_data[col],
                        name=col.replace('capex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[i % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=capex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False
                    ),
                    row=1, col=1
                )
    
    # ---- Gráfico OPEX ----
    if opex_cols and not filtered_df.empty:
        opex_data = filtered_df.groupby('plant_type')[opex_cols].mean().reset_index()
        # Evitar división por cero en opex_pct
        sum_opex_by_plant_type = opex_data.set_index('plant_type').sum(axis=1).replace(0,1)
        opex_pct = opex_data.set_index('plant_type').div(sum_opex_by_plant_type, axis=0) * 100
        
        for i, col in enumerate(opex_cols):
            if not opex_data[col].empty:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                 zip(opex_data[col], opex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=opex_data['plant_type'],
                        y=opex_data[col],
                        name=col.replace('opex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[(i + len(capex_cols)) % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=opex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False
                    ),
                    row=2, col=1
                )
    
    # Actualizar diseño
    fig.update_layout(
        title_text='Composición Detallada de Costos por Tipo de Planta',
        barmode='stack',
        plot_bgcolor='white',
        hovermode='x unified',
        height=900,
        margin=dict(t=100, b=80)
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Planta", row=2, col=1)
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Composición de Costos</h4>
        <p>Esta visualización muestra el desglose detallado de los componentes que conforman los costos 
        de capital (CAPEX) y operación (OPEX) para cada tipo de planta seleccionada.</p>
        
        <p><b>Características:</b></p>
        <ul>
            <li><b>Gráficos separados:</b> CAPEX y OPEX mostrados en gráficos independientes para mayor claridad</li>
            <li><b>Valores absolutos:</b> Eje Y muestra valores en USD</li>
            <li><b>Porcentajes:</b> Cada barra muestra el valor en USD y el porcentaje que representa</li>
            <li><b>Interactividad:</b> Pase el cursor sobre las barras para ver detalles completos</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 4. Tabla Resumen con Explicación (MEJORADA)
def display_summary_table(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data(df) # Asegurar que el dataframe que llega aquí ya esté limpio
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    # MODIFICACIÓN: Mostrar mensaje si el DataFrame filtrado está vacío
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada para la Tabla Resumen.</p>
        </div>
        """))
        return # Salir de la función si no hay datos
        
    # Calcular métricas resumen
    summary = filtered_df.groupby('plant_type').agg({
        'capex_total': ['sum', 'mean'],
        'opex_total': ['sum', 'mean'],
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    })
    
    # Calcular ratios con protección contra división por cero
    summary['capex_per_mw'] = summary[('capex_total', 'sum')] / summary[('capacity_mw', 'sum')].replace(0, 1)
    summary['opex_per_mwh'] = summary[('opex_total', 'sum')] / summary[('net_generation_mwh', 'sum')].replace(0, 1)
    
    # Formatear tabla
    summary.columns = [' '.join(col).strip() for col in summary.columns.values]
    summary = summary[[
        'capacity_mw sum', 
        'net_generation_mwh sum',
        'capex_total sum', 
        'capex_total mean',
        'capex_per_mw',
        'opex_total sum',
        'opex_total mean',
        'opex_per_mwh'
    ]]
    
    # Renombrar columnas
    summary.columns = [
        'Capacidad Total (MW)',
        'Generación Total (MWh)',
        'CAPEX Total (USD)',
        'CAPEX Promedio (USD)',
        'CAPEX por MW (USD/MW)',
        'OPEX Total (USD)',
        'OPEX Promedio (USD)',
        'OPEX por MWh (USD/MWh)'
    ]
    
    # Explicación introductoria
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Tabla Resumen de Costos</h4>
        <p>Esta tabla presenta un resumen consolidado de los costos y métricas clave para los tipos de planta 
        y años seleccionados. Los valores incluyen:</p>
        <ul>
            <li><b>CAPEX Total/Promedio:</b> Inversión total en capital y promedio por planta</li>
            <li><b>CAPEX por MW:</b> Costo de capital por unidad de capacidad instalada</li>
            <li><b>OPEX Total/Promedio:</b> Costos operativos totales y promedio por planta</li>
            <li><b>OPEX por MWh:</b> Costo operativo por unidad de energía generada</li>
        </ul>
    </div>
    """
    
    display(HTML(explanation))
    
    # Mostrar tabla con formato mejorado
    styled_table = summary.style.format({
        'CAPEX Total (USD)': '${:,.0f}',
        'CAPEX Promedio (USD)': '${:,.0f}',
        'CAPEX por MW (USD/MW)': '${:,.0f}',
        'OPEX Total (USD)': '${:,.0f}',
        'OPEX Promedio (USD)': '${:,.0f}',
        'OPEX por MWh (USD/MWh)': '${:,.2f}',
        'Capacidad Total (MW)': '{:,.0f}',
        'Generación Total (MWh)': '{:,.0f}'
    }).background_gradient(cmap='Blues', subset=['CAPEX Total (USD)', 'OPEX Total (USD)'])
    
    # Resaltar métricas clave
    styled_table = styled_table.set_properties(
        subset=['CAPEX por MW (USD/MW)', 'OPEX por MWh (USD/MWh)'],
        **{'background-color': '#f0f8ff', 'font-weight': 'bold'}
    )
    
    display(styled_table.set_caption("Resumen Consolidado de Costos"))

## Interfaz de Control Mejorada (MEJORADA)
def create_advanced_dashboard(df):
    # Preparar datos
    df_clean = prepare_data(df)
    
    # Obtener opciones para los widgets
    # Estas son las variables "all_..." que contienen todas las opciones posibles antes de cualquier filtro
    all_years = sorted(df_clean['report_year'].dt.year.unique())
    all_plant_types = sorted(df_clean['plant_type'].unique())
    
    # Usar 'plant_name_ferc1_clean' para obtener la lista de nombres de planta base para el filtrado interno
    all_plant_names_base = sorted(df_clean['plant_name_ferc1_clean'].unique()) if 'plant_name_ferc1_clean' in df_clean.columns else sorted(df_clean['plant_name_ferc1'].unique())
    
    # Generar 'all_plant_names_with_type' para el selector, que incluye el tipo y es el que se muestra al usuario
    all_plant_names_with_type = sorted(df_clean['plant_name_with_type'].unique()) if 'plant_name_with_type' in df_clean.columns else ['No disponible']

    # NUEVA FUNCIONALIDAD: Obtener todos los rangos de capacidad únicos
    all_capacity_ranges = sorted(df_clean['capacity_mw_range'].unique())
    # Filtra 'Error de cálculo de rango' de las opciones que se mostrarán en el selector inicial.
    all_capacity_ranges = [opt for opt in all_capacity_ranges if opt != 'Error de cálculo de rango']


    # Widgets con opción "Todos"
    year_selector = widgets.SelectMultiple(
        options=['Todos'] + all_years,
        value=['Todos'],
        description='Años:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_type_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_types,
        value=['Todos'],
        description='Tipos de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    # Nuevo: Widget de texto para el buscador de nombres de planta
    plant_name_search_bar = widgets.Text(
        value='',
        placeholder='Buscar nombre de planta...',
        description='Buscar:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    plant_name_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_names_with_type, # Opciones iniciales completas
        value=['Todos'],
        description='Nombre de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    # NUEVO WIDGET: Selector para rangos de capacidad
    capacity_range_selector = widgets.SelectMultiple(
        options=['Todos'] + all_capacity_ranges,
        value=['Todos'],
        description='Rangos de Capacidad (MW):',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    analysis_type = widgets.Dropdown(
        options=[
            ('Evolución de Costos', 'evolution'),
            ('Composición de Costos', 'composition'),
            ('Tabla Resumen', 'summary')
        ],
        value='evolution',
        description='Tipo de Análisis:',
        style={'description_width': 'initial'}
    )
    
    # Función para manejar la selección "Todos"
    def get_selected_options(selected, all_options):
        # Esta función ahora maneja el caso de 'No disponible' que puede venir del selector
        if 'Todos' in selected or len(selected) == 0:
            # Filtrar 'Error de cálculo de rango' si está presente en all_options
            return [opt for opt in all_options if opt not in ['No disponible', 'Error de cálculo de rango', 'nan']] 
        return [opt for opt in selected if opt not in ['No disponible', 'Error de cálculo de rango', 'nan']]


    # Función para extraer solo el nombre base de la planta (sin el tipo entre paréntesis y sin números como (1))
    def extract_plant_name_for_filter(plant_full_name_from_selector):
        if pd.isna(plant_full_name_from_selector) or plant_full_name_from_selector == 'No disponible':
            return 'No disponible' # Asegura que 'No disponible' no cause problemas
        
        # Eliminar el tipo de planta entre paréntesis (ej. ' (Nuclear)')
        name_without_type = plant_full_name_from_selector.split('(')[0].strip()
        
        # Eliminar números entre paréntesis si están en el nombre base (ej. 'Palo Verde (1)')
        name_cleaned = re.sub(r'\s*\(\d+\)\s*', '', name_without_type).strip()
        
        return name_cleaned
    
    # Función para actualizar las opciones de los selectores dependientes (años, tipos de planta y rangos de capacidad)
    def update_dependent_selectors(change):
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        
        # No usamos selected_capacity_ranges_filter aquí para determinar las opciones de los otros selectores,
        # ya que update_dependent_selectors debe recalcular TODAS las opciones para los SELECTORES,
        # y el filtro de rango de capacidad ya afectará a las opciones de las plantas.
        # Las opciones de rango de capacidad se actualizan en base a los filtros de año y tipo de planta, no al revés.

        # Filtrar el DataFrame según los años seleccionados (para plant_type_selector)
        selected_years_dt_filter = [pd.to_datetime(str(year)) for year in selected_years_filter]
        
        filtered_for_dependencies_df = df_clean[df_clean['report_year'].isin(selected_years_dt_filter)] if selected_years_dt_filter else df_clean
        
        # Actualizar opciones de plant_type_selector
        current_plant_type_options = sorted(filtered_for_dependencies_df['plant_type'].unique())
        new_plant_type_options = ['Todos'] + [opt for opt in current_plant_type_options if opt != 'nan'] # Filtrar 'nan'
        current_selection_types = [pt for pt in plant_type_selector.value if pt in new_plant_type_options]
        plant_type_selector.options = new_plant_type_options
        plant_type_selector.value = current_selection_types if current_selection_types else ['Todos']

        # Actualizar opciones de capacity_range_selector
        # Este filtro debe depender del año y tipo de planta
        current_capacity_range_options = sorted(filtered_for_dependencies_df['capacity_mw_range'].unique())
        # Filtra 'Error de cálculo de rango' y 'nan' de las opciones presentadas al usuario.
        current_capacity_range_options = [opt for opt in current_capacity_range_options if opt not in ['Error de cálculo de rango', 'nan']]
        new_capacity_range_options = ['Todos'] + current_capacity_range_options
        current_selection_ranges = [cr for cr in capacity_range_selector.value if cr in new_capacity_range_options]
        capacity_range_selector.options = new_capacity_range_options
        capacity_range_selector.value = current_selection_ranges if current_selection_ranges else ['Todos']
        
        # Vuelve a llamar a update_plant_name_options para refrescar las opciones de planta
        update_plant_name_options(None)


    # Nueva función para actualizar las opciones del selector de nombre de planta
    def update_plant_name_options(change):
        search_term = plant_name_search_bar.value.lower()
        
        # Aplicar el filtro de años y tipos de planta para las opciones de nombre de planta
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        selected_capacity_ranges_filter = get_selected_options(capacity_range_selector.value, all_capacity_ranges) # Obtener los rangos de capacidad seleccionados

        selected_years_dt_filter = [pd.to_datetime(str(year)) for year in selected_years_filter]

        filtered_df_for_plants = df_clean[
            (df_clean['report_year'].isin(selected_years_dt_filter)) & 
            (df_clean['plant_type'].isin(selected_plant_types_filter)) &
            (df_clean['capacity_mw_range'].isin(selected_capacity_ranges_filter)) # Filtro por rango de capacidad
        ]

        if 'plant_name_with_type' in filtered_df_for_plants.columns:
            # Obtener todas las opciones de nombre de planta relevantes para los filtros actuales
            current_relevant_plant_names = sorted(filtered_df_for_plants['plant_name_with_type'].unique())
            
            # Filtrar por el término de búsqueda
            filtered_by_search_plant_names = [
                pn for pn in current_relevant_plant_names
                if search_term in pn.lower()
            ]
            
            # Las nuevas opciones deben ser 'Todos' + las que cumplen la búsqueda
            new_plant_name_options = ['Todos'] + filtered_by_search_plant_names
            
            # Mantener la selección actual si sigue siendo válida
            current_selection_plants = [p for p in plant_name_selector.value if p in new_plant_name_options]
            plant_name_selector.options = new_plant_name_options
            plant_name_selector.value = current_selection_plants if current_selection_plants else ['Todos']
        else:
            plant_name_selector.options = ['No disponible']
            plant_name_selector.value = ['No disponible']
            
    # Asignar las funciones de observación a los widgets
    year_selector.observe(update_dependent_selectors, names='value')
    plant_type_selector.observe(update_dependent_selectors, names='value')
    capacity_range_selector.observe(update_dependent_selectors, names='value') # Nuevo observador
    plant_name_search_bar.observe(update_plant_name_options, names='value')

    # Ejecutar las actualizaciones iniciales para poblar los selectores al cargar el dashboard
    # update_dependent_selectors() llamará a update_plant_name_options()
    update_dependent_selectors(None) 

    # Función principal de actualización para los gráficos y tablas
    def update_analysis(analysis_type_val, selected_years_val, selected_plant_types_val, selected_plants_val, selected_capacity_ranges_val):
        
        # Obtener las opciones finales a usar, aplicando 'Todos'
        years_to_use = get_selected_options(selected_years_val, all_years)
        plant_types_to_use = get_selected_options(selected_plant_types_val, all_plant_types)
        capacity_ranges_to_use = get_selected_options(selected_capacity_ranges_val, all_capacity_ranges) # Obtener los rangos de capacidad seleccionados
        
        # Para las plantas, obtenemos los nombres del selector actual y los limpiamos para el filtro
        # Esto es crucial para que el filtro interno del DataFrame coincida con los nombres de la columna 'plant_name_ferc1_clean'
        selected_plants_from_selector_with_type = get_selected_options(selected_plants_val, plant_name_selector.options)
        
        # Limpiar los nombres de planta para el filtrado, usando la nueva función
        plants_to_use_for_filter = [extract_plant_name_for_filter(p) for p in selected_plants_from_selector_with_type]

        # Quitar duplicados por si acaso el extract_plant_name_for_filter genera los mismos nombres de diferentes variantes
        plants_to_use_for_filter = list(set(plants_to_use_for_filter))

        # Si después de limpiar y consolidar, 'Todos' sigue siendo una opción, usar la lista completa de nombres base
        if 'Todos' in plants_to_use_for_filter or not plants_to_use_for_filter:
            plants_to_use_for_filter = all_plant_names_base
        
        # Las funciones de ploteo ya contienen la lógica de filtrado del DataFrame,
        # así que pasamos los valores ya procesados.
        if analysis_type_val == 'evolution':
            plot_cost_evolution(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type_val == 'composition':
            plot_cost_composition(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type_val == 'summary':
            display_summary_table(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
    
    # Interfaz interactiva
    ui = widgets.VBox([
        widgets.HBox([year_selector, plant_type_selector]),
        widgets.HBox([plant_name_search_bar, plant_name_selector]),
        widgets.HBox([capacity_range_selector]), # Agregamos el selector de rango de capacidad aquí
        analysis_type
    ])
    
    out = widgets.interactive_output(
        update_analysis,
        {
            'analysis_type_val': analysis_type,
            'selected_years_val': year_selector,
            'selected_plant_types_val': plant_type_selector,
            'selected_plants_val': plant_name_selector, 
            'selected_capacity_ranges_val': capacity_range_selector # Pasamos el valor del nuevo selector
        }
    )
    
    display(ui, out)

# Asumiendo que df ya está cargado en tu entorno, puedes ejecutar el dashboard
create_advanced_dashboard(df) 

Output()

In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML
from datetime import datetime
import re # Importar módulo de expresiones regulares

# Configuración inicial
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Paleta de colores
COLOR_PALETTE = px.colors.qualitative.Plotly

## 1. Función para limpiar y preparar datos para df2
def prepare_data_df2(df):
    """
    Prepara los datos de df2 asegurando formatos correctos y valores consistentes
    """
    df = df.copy()
    
    # Convertir año a datetime
    if not pd.api.types.is_datetime64_any_dtype(df['report_year']):
        try:
            df['report_year'] = pd.to_datetime(df['report_year'].astype(int), format='%Y')
        except:
            df['report_year'] = pd.to_datetime(df['report_year'])
    
    # Asegurar costos positivos
    cost_cols = [col for col in df.columns if 'capex_' in col or 'opex_' in col]
    for col in cost_cols:
        if col in df.columns:
            df[col] = df[col].abs()
    
    # Calcular totales si no existen
    if 'capex_total' not in df.columns:
        capex_cols = [col for col in df.columns if col.startswith('capex_') and col != 'capex_total']
        df['capex_total'] = df[capex_cols].sum(axis=1)
    
    if 'opex_production_total' not in df.columns:
        opex_cols = [col for col in df.columns if col.startswith('opex_') and 
                      col not in ['opex_production_total', 'opex_per_mwh', 'opex_transfer']]
        df['opex_production_total'] = df[opex_cols].sum(axis=1)
    
    # Calcular ratios
    df['capex_per_mw'] = df.apply(
        lambda x: x['capex_total'] / x['capacity_mw'] if x['capacity_mw'] > 0 else 0,
        axis=1
    )
    
    df['opex_per_mwh'] = df.apply(
        lambda x: x['opex_production_total'] / x['net_generation_mwh'] if x['net_generation_mwh'] > 0 else 0,
        axis=1
    )
    
    # Asegurar que plant_type sea string
    df['plant_type'] = df['plant_type'].astype(str).replace('nan', 'No especificado')
    
    # Asegurar que plant_name_ferc1 sea string y agregar tipo de planta entre paréntesis
    if 'plant_name_ferc1' in df.columns:
        df['plant_name_ferc1'] = df['plant_name_ferc1'].astype(str).replace('nan', 'No especificado')
        
        # Nuevo: Limpiar plant_name_ferc1 de posibles números entre paréntesis como "(1)"
        # Esto es crucial para que el nombre base coincida con el filtro de planta
        df['plant_name_ferc1_clean'] = df['plant_name_ferc1'].apply(lambda x: re.sub(r'\s*\(\d+\)\s*', '', x).strip())
        
        # Crear columna combinada para mostrar en el selector, usando el nombre limpio
        df['plant_name_with_type'] = df['plant_name_ferc1_clean'] + ' (' + df['plant_type'] + ')'
    
    # NUEVA FUNCIONALIDAD: Crear columna de rangos de capacidad
    # Definir los bins y etiquetas para los rangos de capacidad
    # Ajustar el último bin para asegurar que el valor máximo actual de capacity_mw esté incluido
    max_capacity = df['capacity_mw'].max()
    bins = [0, 10, 50, 100, 250, 500, 1000, 2000, max_capacity + 1] 
    labels = ['0-10 MW', '11-50 MW', '51-100 MW', '101-250 MW', '251-500 MW', '501-1000 MW', '1001-2000 MW', '>2000 MW']
    
    # Crear la nueva columna 'capacity_mw_range'
    df['capacity_mw_range'] = pd.cut(df['capacity_mw'], bins=bins, labels=labels, right=True, include_lowest=True)
    df['capacity_mw_range'] = df['capacity_mw_range'].astype(str).replace('nan', 'No especificado')

    return df

## 2. Evolución de Costos y Capacidad para df2
def plot_cost_evolution_df2(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data_df2(df)
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]
    
    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'

    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    # Agrupar por año
    agg_dict = {
        'capex_total': 'sum',
        'opex_production_total': 'sum',
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }
    
    # Solo agregar opex_fuel si existe en el DataFrame
    if 'opex_fuel' in df_clean.columns:
        agg_dict['opex_fuel'] = 'sum'
    
    yearly_data = filtered_df.groupby('report_year').agg(agg_dict).reset_index()
    
    # Calcular ratios
    yearly_data['capex_per_mw'] = yearly_data['capex_total'] / yearly_data['capacity_mw'].replace(0, 1)
    yearly_data['opex_per_mwh'] = yearly_data['opex_production_total'] / yearly_data['net_generation_mwh'].replace(0, 1)
    
    # Crear figura
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # CAPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['capex_total'],
            name='CAPEX Total (USD)',
            marker_color=COLOR_PALETTE[0],
            opacity=0.7,
            hovertemplate='<b>CAPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX (Barras)
    fig.add_trace(
        go.Bar(
            x=yearly_data['report_year'],
            y=yearly_data['opex_production_total'],
            name='OPEX Total (USD)',
            marker_color=COLOR_PALETTE[1],
            opacity=0.7,
            hovertemplate='<b>OPEX</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
        ),
        secondary_y=False
    )
    
    # OPEX Fuel (Barras) si existe
    if 'opex_fuel' in yearly_data.columns:
        fig.add_trace(
            go.Bar(
                x=yearly_data['report_year'],
                y=yearly_data['opex_fuel'],
                name='Costos Combustible (USD)',
                marker_color=COLOR_PALETTE[2],
                opacity=0.7,
                hovertemplate='<b>Combustible</b><br>Año: %{x|%Y}<br>Total: $%{y:,.0f}<extra></extra>'
            ),
            secondary_y=False
        )
    
    # Capacidad (Línea)
    fig.add_trace(
        go.Scatter(
            x=yearly_data['report_year'],
            y=yearly_data['capacity_mw'],
            name='Capacidad (MW)',
            line=dict(color=COLOR_PALETTE[3], width=3),
            mode='lines+markers',
            hovertemplate='<b>Capacidad</b><br>Año: %{x|%Y}<br>Total: %{y:,.0f} MW<extra></extra>'
        ),
        secondary_y=True
    )
    
    # Actualizar diseño
    fig.update_layout(
        title='Evolución de Costos y Capacidad',
        barmode='group',
        plot_bgcolor='white',
        hovermode='x unified',
        height=500,
        legend=dict(orientation="h", yanchor="bottom", y=1.1, x=0.5, xanchor="center"),
        margin=dict(t=120, b=80), 
        xaxis=dict(tickformat='%Y', type='date')
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="Costos (USD)", secondary_y=False)
    fig.update_yaxes(title_text="Capacidad (MW)", secondary_y=True)
    fig.update_xaxes(title_text="Año")
    
    # Explicación
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Evolución de Costos</h4>
        <p>Este gráfico muestra la evolución temporal de los costos de capital (CAPEX), 
        operación (OPEX) y capacidad instalada.</p>
        <p><b>Nota:</b> Todos los valores de costos han sido convertidos a valores absolutos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 3. Composición de Costos Detallada para df2
def plot_cost_composition_df2(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data_df2(df)
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]

    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    # Componentes CAPEX
    capex_cols = [col for col in df_clean.columns 
                  if col.startswith('capex_') 
                  and col not in ['capex_total', 'capex_per_mw']
                  and col in df_clean.columns
                  and df_clean[col].sum() > 0]
    
    # Componentes OPEX
    opex_cols = [col for col in df_clean.columns 
                 if col.startswith('opex_') 
                 and col not in ['opex_production_total', 'opex_per_mwh', 'opex_transfer']
                 and col in df_clean.columns
                 and df_clean[col].sum() > 0]
    
    # Crear figura con subplots
    fig = make_subplots(rows=2, cols=1, 
                        subplot_titles=("Composición CAPEX (USD)", "Composición OPEX (USD)"),
                        vertical_spacing=0.15)
    
    # ---- Gráfico CAPEX ----
    if capex_cols and not filtered_df.empty: # Añadir comprobación de df vacío
        capex_data = filtered_df.groupby('plant_type')[capex_cols].mean().reset_index()
        capex_pct = capex_data.set_index('plant_type').div(capex_data.set_index('plant_type').sum(axis=1).replace(0,1), axis=0) * 100 # Evitar división por cero
        
        for i, col in enumerate(capex_cols):
            # Solo trazar si hay datos en la columna
            if not capex_data[col].empty and capex_data[col].sum() > 0:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                 zip(capex_data[col], capex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=capex_data['plant_type'],
                        y=capex_data[col],
                        name=col.replace('capex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[i % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=capex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False 
                    ),
                    row=1, col=1
                )
    
    # ---- Gráfico OPEX ----
    if opex_cols and not filtered_df.empty: # Añadir comprobación de df vacío
        opex_data = filtered_df.groupby('plant_type')[opex_cols].mean().reset_index()
        opex_pct = opex_data.set_index('plant_type').div(opex_data.set_index('plant_type').sum(axis=1).replace(0,1), axis=0) * 100 # Evitar división por cero
        
        for i, col in enumerate(opex_cols):
            # Solo trazar si hay datos en la columna
            if not opex_data[col].empty and opex_data[col].sum() > 0:
                text_template = [f"${x:,.0f}<br>({p:.1f}%)" for x, p in 
                                 zip(opex_data[col], opex_pct[col])]
                
                fig.add_trace(
                    go.Bar(
                        x=opex_data['plant_type'],
                        y=opex_data[col],
                        name=col.replace('opex_', '').replace('_', ' ').title(),
                        marker_color=COLOR_PALETTE[(i + len(capex_cols)) % len(COLOR_PALETTE)],
                        text=text_template,
                        textposition='auto',
                        hovertemplate='<b>%{fullData.name}</b><br>Tipo: %{x}<br>Valor: $%{y:,.0f}<br>Porcentaje: %{customdata:.1f}%<extra></extra>',
                        customdata=opex_pct[col],
                        textfont=dict(size=10),
                        showlegend=False 
                    ),
                    row=2, col=1
                )
    
    # Actualizar diseño
    fig.update_layout(
        title_text='Composición Detallada de Costos por Tipo de Planta',
        barmode='stack',
        plot_bgcolor='white',
        hovermode='x unified',
        height=900,
        margin=dict(t=100, b=80) 
    )
    
    # Configurar ejes
    fig.update_yaxes(title_text="USD", row=1, col=1)
    fig.update_yaxes(title_text="USD", row=2, col=1)
    fig.update_xaxes(title_text="Tipo de Planta", row=2, col=1)
    
    # Explicación
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Análisis de Composición de Costos</h4>
        <p>Esta visualización muestra el desglose detallado de los componentes que conforman los costos 
        de capital (CAPEX) y operación (OPEX) para cada tipo de planta seleccionada.</p>
        <p><b>Interactividad:</b> Pase el cursor sobre las barras para ver detalles completos.</p>
    </div>
    """
    
    display(HTML(explanation))
    fig.show()

## 4. Tabla Resumen para df2
def display_summary_table_df2(df, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
    df_clean = prepare_data_df2(df)
    
    # Asegurar que los filtros sean listas y que el 'Todos' no esté presente en la comparación si es el caso
    years_to_filter = [y for y in selected_years if y != 'Todos'] if 'Todos' in selected_years else selected_years
    plant_types_to_filter = [pt for pt in selected_plant_types if pt != 'Todos'] if 'Todos' in selected_plant_types else selected_plant_types
    plants_to_filter = [p for p in selected_plants if p != 'Todos'] if 'Todos' in selected_plants else selected_plants
    capacity_ranges_to_filter = [cr for cr in selected_capacity_ranges if cr != 'Todos'] if 'Todos' in selected_capacity_ranges else selected_capacity_ranges

    selected_years_dt = [pd.to_datetime(str(year)) for year in years_to_filter]

    # Usar 'plant_name_ferc1_clean' para el filtrado si existe, de lo contrario 'plant_name_ferc1'
    plant_name_col_to_filter = 'plant_name_ferc1_clean' if 'plant_name_ferc1_clean' in df_clean.columns else 'plant_name_ferc1'
    
    filtered_df = df_clean[
        (df_clean['report_year'].isin(selected_years_dt) if years_to_filter else df_clean['report_year'].isin(df_clean['report_year'].unique())) & 
        (df_clean['plant_type'].isin(plant_types_to_filter) if plant_types_to_filter else df_clean['plant_type'].isin(df_clean['plant_type'].unique())) &
        (df_clean[plant_name_col_to_filter].isin(plants_to_filter) if plants_to_filter and plant_name_col_to_filter in df_clean.columns else pd.Series([True]*len(df_clean))) &
        (df_clean['capacity_mw_range'].isin(capacity_ranges_to_filter) if capacity_ranges_to_filter else pd.Series([True]*len(df_clean))) # Nuevo filtro por rango de capacidad
    ]
    
    # MODIFICACIÓN: Mostrar mensaje si el DataFrame filtrado está vacío
    if filtered_df.empty:
        display(HTML("""
        <div style="background-color:#fff3cd; padding:15px; border-radius:5px; margin-bottom:20px; border: 1px solid #ffeeba;">
            <p style="color:#856404; margin:0;">No hay datos disponibles para la combinación de filtros seleccionada.</p>
        </div>
        """))
        return 
        
    # Calcular métricas resumen
    agg_dict = {
        'capex_total': ['sum', 'mean'],
        'opex_production_total': ['sum', 'mean'],
        'capacity_mw': 'sum',
        'net_generation_mwh': 'sum'
    }
    
    # Agregar opex_fuel si existe
    if 'opex_fuel' in df_clean.columns:
        agg_dict['opex_fuel'] = ['sum', 'mean']
    
    summary = filtered_df.groupby('plant_type').agg(agg_dict)
    
    # Calcular ratios
    summary['capex_per_mw'] = summary[('capex_total', 'sum')] / summary[('capacity_mw', 'sum')].replace(0, 1)
    summary['opex_per_mwh'] = summary[('opex_production_total', 'sum')] / summary[('net_generation_mwh', 'sum')].replace(0, 1)
    
    # Formatear tabla
    summary.columns = [' '.join(col).strip() for col in summary.columns.values]
    
    # Seleccionar y renombrar columnas
    columns_to_show = [
        'capacity_mw sum', 
        'net_generation_mwh sum',
        'capex_total sum', 
        'capex_total mean',
        'capex_per_mw',
        'opex_production_total sum',
        'opex_production_total mean',
        'opex_per_mwh'
    ]
    
    if 'opex_fuel sum' in summary.columns:
        columns_to_show.extend([
            'opex_fuel sum',
            'opex_fuel mean'
        ])
    
    summary = summary[columns_to_show]
    
    # Renombrar columnas
    new_names = {
        'capacity_mw sum': 'Capacidad Total (MW)',
        'net_generation_mwh sum': 'Generación Total (MWh)',
        'capex_total sum': 'CAPEX Total (USD)',
        'capex_total mean': 'CAPEX Promedio (USD)',
        'capex_per_mw': 'CAPEX por MW (USD/MW)',
        'opex_production_total sum': 'OPEX Total (USD)',
        'opex_production_total mean': 'OPEX Promedio (USD)',
        'opex_per_mwh': 'OPEX por MWh (USD/MWh)',
        'opex_fuel sum': 'Costos Combustible Total (USD)',
        'opex_fuel mean': 'Costos Combustible Promedio (USD)'
    }
    
    summary.columns = [new_names.get(col, col) for col in summary.columns]
    
    # Explicación
    explanation = """
    <div style="background-color:#f8f9fa; padding:15px; border-radius:5px; margin-bottom:20px">
        <h4>Tabla Resumen de Costos y Métricas</h4>
        <p>Esta tabla presenta un resumen consolidado de los costos y métricas clave para los tipos de planta 
        y años seleccionados.</p>
    </div>
    """
    
    display(HTML(explanation))
    
    # Mostrar tabla con formato
    format_dict = {
        'CAPEX Total (USD)': '${:,.0f}',
        'CAPEX Promedio (USD)': '${:,.0f}',
        'CAPEX por MW (USD/MW)': '${:,.0f}',
        'OPEX Total (USD)': '${:,.0f}',
        'OPEX Promedio (USD)': '${:,.0f}',
        'OPEX por MWh (USD/MWh)': '${:,.2f}',
        'Capacidad Total (MW)': '{:,.0f}',
        'Generación Total (MWh)': '{:,.0f}'
    }
    
    if 'Costos Combustible Total (USD)' in summary.columns:
        format_dict.update({
            'Costos Combustible Total (USD)': '${:,.0f}',
            'Costos Combustible Promedio (USD)': '${:,.0f}'
        })
    
    styled_table = summary.style.format(format_dict).background_gradient(
        cmap='Blues', 
        subset=[col for col in summary.columns if 'USD' in col and 'por' not in col]
    )
    
    styled_table = styled_table.set_properties(
        subset=[col for col in summary.columns if 'por' in col or 'por' in col],
        **{'background-color': '#f0f8ff', 'font-weight': 'bold'}
    )
    
    display(styled_table.set_caption("Resumen Consolidado de Costos y Métricas"))

## Interfaz de Control para df2
def create_advanced_dashboard_df2(df):
    # Preparar datos
    df_clean = prepare_data_df2(df)
    
    # Obtener opciones iniciales para los widgets
    # Estas son las variables "all_..." que contienen todas las opciones posibles
    all_years = sorted(df_clean['report_year'].dt.year.unique())
    all_plant_types = sorted(df_clean['plant_type'].unique())
    
    # Usar 'plant_name_ferc1_clean' para obtener la lista de nombres de planta base
    all_plant_names_base = sorted(df_clean['plant_name_ferc1_clean'].unique()) if 'plant_name_ferc1_clean' in df_clean.columns else sorted(df_clean['plant_name_ferc1'].unique())
    # Generar 'all_plant_names_with_type' para el selector, pero el filtro usará 'plant_name_ferc1_clean'
    all_plant_names_with_type = sorted(df_clean['plant_name_with_type'].unique()) if 'plant_name_with_type' in df_clean.columns else ['No disponible']
    
    # NUEVA FUNCIONALIDAD: Obtener todos los rangos de capacidad únicos
    all_capacity_ranges = sorted(df_clean['capacity_mw_range'].unique())

    # Widgets
    year_selector = widgets.SelectMultiple(
        options=['Todos'] + all_years,
        value=['Todos'],
        description='Años:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    plant_type_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_types,
        value=['Todos'],
        description='Tipos de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    # MODIFICACIÓN 2: Añadir un widget de texto para el buscador
    plant_name_search_bar = widgets.Text(
        value='',
        placeholder='Buscar nombre de planta...',
        description='Buscar:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    plant_name_selector = widgets.SelectMultiple(
        options=['Todos'] + all_plant_names_with_type, # Opciones iniciales completas
        value=['Todos'],
        description='Nombre de Planta:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )

    # NUEVO WIDGET: Selector para rangos de capacidad
    capacity_range_selector = widgets.SelectMultiple(
        options=['Todos'] + all_capacity_ranges,
        value=['Todos'],
        description='Rangos de Capacidad (MW):',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '300px'}
    )
    
    analysis_type = widgets.Dropdown(
        options=[
            ('Evolución de Costos', 'evolution'),
            ('Composición de Costos', 'composition'),
            ('Tabla Resumen', 'summary')
        ],
        value='evolution',
        description='Tipo de Análisis:',
        style={'description_width': 'initial'}
    )
    
    # Función para manejar selección "Todos"
    def get_selected_options(selected, all_options):
        if 'Todos' in selected or len(selected) == 0:
            return all_options
        return selected
    
    # Función para extraer solo el nombre base de la planta (sin el tipo entre paréntesis y sin números como (1))
    def extract_plant_name(plant_full_name):
        if pd.isna(plant_full_name): # Manejar NaN
            return 'No especificado'
        name_without_type = plant_full_name.split('(')[0].strip()
        # Eliminar números entre paréntesis si están en el nombre base (ej. Palo Verde (1))
        name_cleaned = re.sub(r'\s*\(\d+\)\s*', '', name_without_type).strip()
        return name_cleaned
    
    # Función para actualizar las opciones de los selectores dependientes (años, tipos de planta y rangos de capacidad)
    def update_dependent_selectors(change):
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        selected_capacity_ranges_filter = get_selected_options(capacity_range_selector.value, all_capacity_ranges)

        # Filtrar el DataFrame según los años y tipos de planta seleccionados
        filtered_for_dependencies_df = df_clean[
            (df_clean['report_year'].dt.year.isin(selected_years_filter)) & 
            (df_clean['plant_type'].isin(selected_plant_types_filter))
        ]
        # Aplica el filtro de rango de capacidad para actualizar las opciones de nombre de planta
        filtered_for_plants_df = filtered_for_dependencies_df[
            (filtered_for_dependencies_df['capacity_mw_range'].isin(selected_capacity_ranges_filter))
        ]

        # Actualizar opciones de plant_type_selector
        current_plant_type_options = sorted(filtered_for_dependencies_df['plant_type'].unique())
        new_plant_type_options = ['Todos'] + current_plant_type_options
        current_selection_types = [pt for pt in plant_type_selector.value if pt in new_plant_type_options]
        plant_type_selector.options = new_plant_type_options
        plant_type_selector.value = current_selection_types if current_selection_types else ['Todos']

        # Actualizar opciones de capacity_range_selector
        current_capacity_range_options = sorted(filtered_for_dependencies_df['capacity_mw_range'].unique())
        new_capacity_range_options = ['Todos'] + current_capacity_range_options
        current_selection_ranges = [cr for cr in capacity_range_selector.value if cr in new_capacity_range_options]
        capacity_range_selector.options = new_capacity_range_options
        capacity_range_selector.value = current_selection_ranges if current_selection_ranges else ['Todos']
        
        # Vuelve a llamar a update_plant_name_options después de actualizar los otros selectores
        update_plant_name_options(None)


    # MODIFICACIÓN 2: Nueva función para actualizar las opciones del selector de nombre de planta
    def update_plant_name_options(change):
        search_term = plant_name_search_bar.value.lower()
        
        # Aplicar todos los filtros para determinar las opciones disponibles de nombre de planta
        selected_years_filter = get_selected_options(year_selector.value, all_years)
        selected_plant_types_filter = get_selected_options(plant_type_selector.value, all_plant_types)
        selected_capacity_ranges_filter = get_selected_options(capacity_range_selector.value, all_capacity_ranges)

        filtered_df_for_plants = df_clean[
            (df_clean['report_year'].dt.year.isin(selected_years_filter)) & 
            (df_clean['plant_type'].isin(selected_plant_types_filter)) &
            (df_clean['capacity_mw_range'].isin(selected_capacity_ranges_filter)) # Filtro por rango de capacidad
        ]

        if 'plant_name_with_type' in filtered_df_for_plants.columns:
            # Filtrar por el término de búsqueda en 'plant_name_with_type'
            filtered_plant_names = [
                pn for pn in sorted(filtered_df_for_plants['plant_name_with_type'].unique()) 
                if search_term in pn.lower()
            ]
            
            new_plant_name_options = ['Todos'] + filtered_plant_names
            
            # Si el término de búsqueda está vacío, mostrar todas las opciones relevantes (filtradas por año/tipo/rango)
            if not search_term:
                new_plant_name_options = ['Todos'] + sorted(filtered_df_for_plants['plant_name_with_type'].unique())

            # Mantener la selección actual si sigue siendo válida en las nuevas opciones
            current_selection_plants = [p for p in plant_name_selector.value if p in new_plant_name_options]
            plant_name_selector.options = new_plant_name_options
            plant_name_selector.value = current_selection_plants if current_selection_plants else ['Todos']
        else:
            plant_name_selector.options = ['No disponible']
            plant_name_selector.value = ['No disponible']
            
    # Asignar la función de observación a los selectores de año, tipo de planta y rango de capacidad
    year_selector.observe(update_dependent_selectors, names='value')
    plant_type_selector.observe(update_dependent_selectors, names='value')
    capacity_range_selector.observe(update_dependent_selectors, names='value') # Nuevo observador
    # MODIFICACIÓN 2: Asignar la función de observación a la barra de búsqueda
    plant_name_search_bar.observe(update_plant_name_options, names='value')

    # Ejecutar las actualizaciones iniciales para que los filtros se muestren correctamente al cargar
    update_dependent_selectors(None) 

    # Función de actualización para los gráficos y tablas
    def update_analysis(analysis_type, selected_years, selected_plant_types, selected_plants, selected_capacity_ranges):
        # Aquí se usan las variables "all_..." que están definidas
        # al inicio de create_advanced_dashboard_df2.
        years_to_use = get_selected_options(selected_years, all_years)
        plant_types_to_use = get_selected_options(selected_plant_types, all_plant_types)
        capacity_ranges_to_use = get_selected_options(selected_capacity_ranges, all_capacity_ranges) # Obtener los rangos de capacidad seleccionados
        
        # Para las plantas, necesitamos la lista de los nombres de planta *limpios* que se usarán en el filtro.
        # Esta lista debe derivarse de las opciones *actualmente disponibles en el selector*
        # pero extrayendo el nombre base sin el "(tipo)" ni "(número)".
        current_selectable_plant_names_with_type = get_selected_options(selected_plants, plant_name_selector.options)
        plants_to_use_for_filter = [extract_plant_name(p) for p in current_selectable_plant_names_with_type if p != 'No disponible']
        
        # Asegurarse de no pasar 'Todos' a las funciones de plot/summary si se seleccionó para el selector
        if 'Todos' in plants_to_use_for_filter:
            plants_to_use_for_filter = all_plant_names_base # Usa la lista base limpia de todas las plantas

        # Las funciones de ploteo ya contienen la lógica de filtrado del DataFrame,
        # así que pasamos los valores directamente.
        if analysis_type == 'evolution':
            plot_cost_evolution_df2(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type == 'composition':
            plot_cost_composition_df2(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
        elif analysis_type == 'summary':
            display_summary_table_df2(df_clean, years_to_use, plant_types_to_use, plants_to_use_for_filter, capacity_ranges_to_use)
    
    # Interfaz interactiva
    ui = widgets.VBox([
        widgets.HBox([year_selector, plant_type_selector]),
        widgets.HBox([plant_name_search_bar, plant_name_selector]),
        widgets.HBox([capacity_range_selector]), # Agregamos el selector de rango de capacidad aquí
        analysis_type
    ])
    
    out = widgets.interactive_output(
        update_analysis,
        {
            'analysis_type': analysis_type,
            'selected_years': year_selector,
            'selected_plant_types': plant_type_selector,
            'selected_plants': plant_name_selector, 
            'selected_capacity_ranges': capacity_range_selector # Pasamos el valor del nuevo selector
        }
    )
    
    display(ui, out)

# Asumiendo que df2 ya está cargado en tu entorno, puedes ejecutar el dashboard
create_advanced_dashboard_df2(df2)

Output()